In [61]:
import skimage.io as io
from keras.layers import BatchNormalization
from skimage import img_as_uint
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import *
from keras.callbacks import *
from tensorflow.python.keras.optimizer_v2.adam import Adam

In [62]:
def FCN_model(pretrained_weights = None, input_size = (None,None,3)):
    inputs = Input(shape=input_size)

    x = Conv2D(filters=64, kernel_size=1, strides=1)(inputs)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Fully connected layer 2
    x = Conv2D(filters=32, kernel_size=1, strides=1)(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)

    x = Conv2D(filters=16, kernel_size=1, strides=1)(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)

    x = Conv2D(filters=8, kernel_size=1, strides=1)(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = GlobalMaxPooling2D()(x)

    predictions = Activation('softmax')(x)

    model = Model(inputs, predictions)

    model.compile(optimizer=Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    if pretrained_weights:
        model.load_weights(pretrained_weights)
    return model

def unet(pretrained_weights = None,input_size = (None,None,3)):
    inputs = Input(input_size)
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    drop3 = Dropout(0.5)(conv3)

    conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)

    up5 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop4))
    merge5 = concatenate([drop3,up5], axis = 3)
    conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge5)
    conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)

    up6 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv5))
    merge6 = concatenate([conv2,up6], axis = 3)
    conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv1,up7], axis = 3)
    conv7 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    conv8 = Conv2D(1, 1, activation = 'sigmoid')(conv7)
    model = Model(inputs, conv8)
    model.compile(optimizer=Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

    if pretrained_weights:
        model.load_weights(pretrained_weights)
    return model

In [63]:
def adjustData(img,mask):
    img = img / 255
    mask = mask /255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0

    return (img,mask)

def testGenerator(train_path):
    for i in range(1, 15):
        img = io.imread(os.path.join(train_path,"%d.jpg"%i),as_gray = True)
        img /= 255.0 -0.5
        img = np.array(img, dtype=np.uint8)
        img = np.reshape(img,img.shape+(1,))
        img = np.reshape(img,(1,)+img.shape)
        yield img

def geneTrainNpy(image_path,mask_path):
    image_files = [f for f in listdir(image_path) if isfile(join(image_path, f))]
    mask_files = [f for f in listdir(mask_path) if isfile(join(mask_path, f))]
    image_arr = []
    mask_arr = []
    for i in range(len(image_files)):
        img = io.imread(os.path.join(image_path, image_files[i]))
        mask = io.imread(os.path.join(mask_path, mask_files[i]))
        img = np.reshape(img,img.shape)
        mask = np.reshape(mask,mask.shape)
        img,mask = adjustData(img,mask)
        image_arr.append(img)
        mask_arr.append(mask)
    image_arr = np.array(image_arr)
    mask_arr = np.array(mask_arr)
    return image_arr, mask_arr

def saveResult(save_path, npyfile):
    for i,item in enumerate(npyfile):
        plt.imshow(item, interpolation='nearest')
        plt.show()
        item /= 255. - 0.5
        io.imsave(os.path.join(save_path,"%d_predict.tif"%i),img_as_uint(item))

In [64]:
def train_model():
    model_checkpoint = ModelCheckpoint('unet_eye.hdf5', monitor='loss',verbose=1, save_best_only=True)
    model = FCN_model()
    imgs_train,imgs_mask_train = geneTrainNpy("eye","vessels")
    batch_size = 2
    epochs = 1
    steps_per_epoch = len(imgs_train) // batch_size
    model.fit(imgs_train,imgs_mask_train, batch_size=batch_size,
                  epochs=epochs, steps_per_epoch=steps_per_epoch,
                  verbose=1,validation_split=0.2, shuffle=True,
                  callbacks=[model_checkpoint])


def test_model():
    model = unet("unet_eye.hdf5")
    testGene = testGenerator("eye")
    results = model.predict(testGene, size = 15,verbose=1)
    saveResult("results",results)


In [65]:
#train_model()
#test_model()

In [66]:

train_model()

The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_24), but
are not present in its tracked objects:
  <tf.Variable 'batch_normalization_24/gamma:0' shape=(64,) dtype=float32>
  <tf.Variable 'batch_normalization_24/beta:0' shape=(64,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_25), but
are not present in its tracked objects:
  <tf.Variable 'batch_normalization_25/gamma:0' shape=(32,) dtype=float32>
  <tf.Variable 'batch_normalization_25/beta:0' shape=(32,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were u

ValueError: in user code:


    ValueError: `logits` and `labels` must have the same shape, received ((2, 8) vs (2, 2336, 3504)).
